# Sora with Azure AI Foundry

Sora is an AI model from OpenAI that can create realistic and imaginative video scenes from text instructions. The model is capable of generating a wide range of video content, including realistic scenes, animations, and special effects. Several video resolutions and durations are supported.

https://learn.microsoft.com/en-us/azure/ai-services/openai/concepts/video-generation

In [ ]:
%pip install python_dotenv

In [ ]:
import base64
import datetime
import os
import requests
import sys
import time

from dotenv import load_dotenv
from IPython.display import Video, FileLink
from moviepy import *

In [ ]:
sys.version

In [ ]:
print(f"Today is {datetime.datetime.today().strftime('%d-%b-%Y %H:%M:%S')}")

## Settings

In [ ]:
OUTPUT_DIR = "videos"

os.makedirs(OUTPUT_DIR, exist_ok=True)

In [ ]:
load_dotenv("azure.env")

endpoint = os.environ['AZURE_OPENAI_ENDPOINT']
api_key = os.environ['AZURE_OPENAI_API_KEY']

model = "sora"

# Function

In [ ]:
def sora(prompt, width=480, height=480, n_seconds=5):
    """
    Generates a video based on the given prompt using the SORA model.

    Parameters:
    prompt (str): The text prompt to generate the video.
    width (int): The width of the video. Supported values are 480, 854, 720, 1080, and 1920.
    height (int): The height of the video. Supported values are 480, 854, 720, 1080, and 1920.
    n_seconds (int): The duration of the video in seconds. Must be between 1 and 20 seconds.
    n_variants (int): The number of video variants to generate.
    
    Returns:
    str: The filename of the generated video.

    Raises:
    Exception: If the video generation job fails or no generations are found.
    """
    start = time.time()

    api_version = 'preview'
    headers = {"api-key": api_key, "Content-Type": "application/json"}

    idx = datetime.datetime.today().strftime('%d%b%Y_%H%M%S')
    suffix = prompt[:30].replace(",", "_").replace(".", "_").replace(" ", "_")
    output_filename = os.path.join(OUTPUT_DIR, f"sora_{idx}_{suffix}.mp4")

    # 1. Create a video generation job
    create_url = f"{endpoint}/openai/v1/video/generations/jobs?api-version={api_version}"
    
    body = {
        "prompt": prompt,
        "width": width,  # 480x480, 480x854, 854x480, 720x720, 720x1280, 1280x720, 1080x1080, 1080x1920, 1920x1080.
        "height": height,  # 480x480, 480x854, 854x480, 720x720, 720x1280, 1280x720, 1080x1080, 1080x1920, 1920x1080.
        "n_seconds": n_seconds,  # between 1 and 20 seconds
        "model": model,  # SORA model
    }
    response = requests.post(create_url, headers=headers, json=body)
    response.raise_for_status()

    now = datetime.datetime.today().strftime('%d-%b-%Y %H:%M:%S')
    print(f"{now} Full response JSON:", response.json())
    print()

    job_id = response.json()["id"]
    now = datetime.datetime.today().strftime('%d-%b-%Y %H:%M:%S')
    print(f"{now} Job created: {job_id}")

    # 2. Poll for job status
    status_url = f"{endpoint}/openai/v1/video/generations/jobs/{job_id}?api-version={api_version}"
    status = None

    while status not in ("succeeded", "failed", "cancelled"):
        time.sleep(5)  # Wait before polling again
        status_response = requests.get(status_url, headers=headers).json()
        status = status_response.get("status")
        now = datetime.datetime.today().strftime('%d-%b-%Y %H:%M:%S')
        print(f"{now} Job status: {status}")

    # 3. Retrieve generated video
    if status == "succeeded":
        generations = status_response.get("generations", [])

        if generations:
            now = datetime.datetime.today().strftime('%d-%b-%Y %H:%M:%S')
            print(f"\n{now} ✅ Done. Video generation succeeded.")
            generation_id = generations[0].get("id")
            video_url = f"{endpoint}/openai/v1/video/generations/{generation_id}/content/video?api-version={api_version}"
            video_response = requests.get(video_url, headers=headers)

            if video_response.ok:
                # Downloading the video
                print("\nDownloading the video...")
                with open(output_filename, "wb") as file:
                    file.write(video_response.content)
                    print(f"SORA Generated video saved: '{output_filename}'")

                elapsed = time.time() - start
                minutes, seconds = divmod(elapsed, 60)
                print(f"Done in {minutes:.0f} minutes and {seconds:.0f} seconds")

                return output_filename
        else:
            raise Exception("Error. No generations found in job result.")
    else:
        raise Exception(f"Error. Job did not succeed. Status: {status}")

## Example

In [ ]:
prompt = "A young boy and his father playing together in the ocean on the beach."

generated_video = sora(prompt, width=480, height=480, n_seconds=5)

In [ ]:
Video(generated_video)

In [ ]:
video_link = FileLink(path=generated_video)
video_link

## Another example

In [ ]:
prompt = "A close up view of a glass sphere that has a zen garden within it. There is a small dwarf in the sphere who is raking the zen garden and creating patterns in the sand."

generated_video = sora(prompt, width=480, height=480, n_seconds=5)

In [ ]:
Video(generated_video)

In [ ]:
video_link = FileLink(path=generated_video)
video_link

## Another example

In [ ]:
prompt = "Several giant wooly mammoths approach treading through a snowy meadow, their long wooly fur lightly blows in the wind as they walk, snow covered trees and dramatic snow capped mountains in the distance, mid afternoon light with wispy clouds and a sun high in the distance creates a warm glow, the low camera view is stunning capturing the large furry mammal with beautiful photography, depth of field."

generated_video = sora(prompt,
                       width=1280,
                       height=720,
                       n_seconds=5,
                       )

In [ ]:
Video(generated_video, width=640)

In [ ]:
video_link = FileLink(path=generated_video)
video_link

## Another example

In [ ]:
prompt = "Drone view of waves crashing against the rugged cliffs along Big Sur's garay point beach. The crashing blue waters create white-tipped waves, while the golden light of the setting sun illuminates the rocky shore. A small island with a lighthouse sits in the distance, and green shrubbery covers the cliff's edge. The steep drop from the road down to the beach is a dramatic feat, with the cliff's edges jutting out over the sea. This is a view that captures the raw beauty of the coast and the rugged landscape of the Pacific Coast Highway."

generated_video = sora(prompt,
                       width=1280,
                       height=720,
                       n_seconds=15,
                       )

In [ ]:
Video(generated_video)

In [ ]:
video_link = FileLink(path=generated_video)
video_link

## Another example

In [ ]:
prompt = "Extreme close up of a 25 year old woman's eye blinking, cinematic film shot in 70mm, depth of field, vivid colors, cinematic"

generated_video = sora(prompt,
                       width=1280,
                       height=720,
                       n_seconds=5,
                       )

In [ ]:
Video(generated_video, width=860)

In [ ]:
video_link = FileLink(path=generated_video)
video_link

## Another example

In [ ]:
prompt = "Extreme close up of a 50 year old male's eye blinking, cinematic film shot in 70mm, depth of field, vivid colors, cinematic"

generated_video = sora(prompt,
                       width=1280,
                       height=720,
                       n_seconds=5,
                       )

In [ ]:
Video(generated_video, width=860)

In [ ]:
video_link = FileLink(path=generated_video)
video_link

## Another example

In [ ]:
prompt = "Reflections in the window of a train traveling through the Tokyo suburbs."

generated_video = sora(prompt,
                       width=1280,
                       height=720,
                       n_seconds=10,
                       )

In [ ]:
Video(generated_video, width=860)

In [ ]:
video_link = FileLink(path=generated_video)
video_link

## Another example

In [ ]:
prompt = "A drone view of flock of paper airplanes multicolors flutters through a dense jungle, weaving around trees as if they were migrating birds."

generated_video = sora(prompt,
                       width=1280,
                       height=720,
                       n_seconds=7,
                       )

In [ ]:
Video(generated_video, width=860)

In [ ]:
video_link = FileLink(path=generated_video)
video_link

## Another exemple

In [ ]:
prompt = "Street of Tokyo, rainy day with people walking in a street, restaurants in the street."

generated_video = sora(prompt,
                       width=1280,
                       height=720,
                       n_seconds=7,
                       )

In [ ]:
Video(generated_video, width=860)

In [ ]:
video_link = FileLink(path=generated_video)
video_link

In [ ]:
!ls $OUTPUT_DIR -lh